In [41]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 2.1 MB/s 
     |████████████████████████████████| 163 kB 66.4 MB/s 
     |████████████████████████████████| 6.6 MB 47.4 MB/s 


In [42]:
import os
import random

import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel, pipeline

In [43]:
def load_model():
    tokenizer = AutoTokenizer.from_pretrained('youscan/ukr-roberta-base')
    model = AutoModel.from_pretrained('youscan/ukr-roberta-base')
    return tokenizer, model

tokenizer, model = load_model()

Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/507M [00:00<?, ?B/s]

Some weights of the model checkpoint at youscan/ukr-roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [44]:
def model_embedding(text, tokenizer, model):
    input_ids = tokenizer(text, return_tensors="pt")["input_ids"]
    outputs = model(input_ids, output_hidden_states=True)
    emb = outputs[2]
    emb = np.array([i[0].detach().numpy() for i in emb])[:]
    emb = emb.mean(axis=(0, 1))
    return emb

In [56]:
def load_data():
    data = pd.read_csv('train_small2.csv', index_col=0)
    data['text'] = data['text'].str.lower()
    return data

def train_test_split(data):
    train_data = data[data['target_numerical'] != 2].copy()
    test_data = data[data['target_numerical'] == 2].copy()
    return train_data, test_data

data = load_data()
train_data, test_data = train_test_split(data)

In [58]:
labels_list = train_data['target'].values
descriptions_list = train_data['target'].values
embeddings_list = {}

In [49]:
for label, descr in zip(labels_list, descriptions_list):
    tmp = embeddings_list.get(label, [])
    tmp.append(model_embedding(descr, tokenizer, model))
    embeddings_list[label] = tmp

In [50]:
for label, embeddings in embeddings_list.items():
    embeddings_list[label] = np.mean(embeddings, axis=0)

In [60]:
test_labels_list = test_data['target'].values
test_descriptions_list = test_data['text'].values

In [52]:
def distance(a, b):
    return sum([(i - j) ** 2 for i, j in zip(a, b)]) ** .5

In [64]:
t = 0
pred_labels = []
for ind in range(len(test_descriptions_list)):

    label = test_labels_list[ind]
    descr = test_descriptions_list[ind]
    print(f'LABEL {label}\tDESCR: {descr}')

    test_emb = model_embedding(descr, tokenizer, model)

    scores = list((distance(el, test_emb), k) for k, el in embeddings_list.items())
    if scores:
      sorted_scores = sorted(scores, key=lambda x: x[0])
      best_preds = sorted_scores[0][1]
      print(label, best_preds)
      pred_labels.append(best_preds)
      if label == best_preds:
          t += 1
        
print(t / len(test_descriptions_list))

LABEL спорт	DESCR: головний тренер солігорського «шахтаря» юрій вернидуб на післяматчевій прес-конференції пояснив, чому його підопічні програли команді «слуцьк» (1:2): «такого початку я не пам'ятаю у своєму житті. але що є, то є. виправдовуватися я не буду. якщо команда в тій грі – зі «смолевичами» – завдає 21 удару, зараз – 24 або 25 ударів, з них 15 у ворота, і ми забити не можемо. не знаю. значить, не вміємо ми забивати. якщо пенальті ми не можемо реалізувати. забивай, і було б 40 хвилин часу. все могло б бути по-іншому. але що маємо, те маємо. у заявці не було сачівка, він травму отримав ще в грі зі «смолевичами». у нього міжреберна невралгія. не міг вийти. у шіндагорідзе проблема з хрестоподібною зв'язкою. коли це вирішиться, я навіть не знаю. треба ще робити повторне мрт. а так як, я так зрозумів, у нього вже була така проблема, то треба їхати туди, де робили. але зараз все закрито – проблема. чого не вистачило в обороні? у першому голі просто йшла подача – і все. до цього були 